In [1]:
import pickle
import gzip

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from util import read_records

In [4]:
r = read_records('../processed-data/records0_train.pkl.gz')

In [5]:
len(r)

31633

In [6]:
r[10]

defaultdict(str,
            {'Id': '<21247412.1075859936765.JavaMail.evans@thyme>',
             'From': 'greg.johnston@enron.com',
             'To': 'dianne.seib@enron.com',
             'Date': 'Tue, 27 Jun 2000 09:22:00',
             'Subject': 'Outstanding Duke EnronOnline Transactions',
             'Body': 'Dianne, further to our conversation this morning, this e-mail will confirm \nthe following:\n\n1.  The current form of Master Physical Gas Agreement with Duke is an older \nmaster which indicates that transactions must be confirmed in writing and \narguably provides that such confirms are not binding until executed by both \nparties.  However, the EnronOnline Electronic Trading Agreement indicates \nthat, where a confirm is issued, it is deemed accepted unless objected to \nwithin three business days.  There is no indication in the EOL Electronic \nTrading Agreement how a conflict between the provisions of that agreement and \nany master agreement will be resolved, but I th

In [7]:
[rec['Label'] for rec in r if rec['From'] == None]

[]

In [8]:
len(r)

31633

In [9]:
import collections
import itertools
import random

In [10]:
def shuffle_generator(gen, shuffle_len=1000):

    buf = collections.deque(itertools.islice(gen, shuffle_len))
    while (len(buf) > 0):
        # shuffle the buffer, by shifting elements a random amount
        k = random.randint(-len(buf)//2, len(buf)//2) 
        buf.rotate(k)

        cur = buf.popleft()
        nxt = next(gen, None)
        if nxt is not None:
            buf.append(nxt);
        yield cur

In [11]:
def test_shuffle_gen(n=1000):
    g = (i for i in range(n))
    for i in shuffle_generator(g):
        pass
def test_gen(n=1000):
    g = (i for i in range(n))
    for i in g:
        pass

In [59]:
%timeit test_shuffle_gen(10000)

28.2 ms ± 448 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
%timeit test_gen(10000)

733 µs ± 5.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
import numpy as np
import tensorflow as tf
import os
DIRECTORY = os.path.abspath('../processed-data/tfrecords-2')
from tfdata_helpers import *

In [23]:
from collections import Counter

In [27]:
filenames = ['{}/dev0.tfrecords'.format(DIRECTORY)]
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parse_tfrecord)

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

counts = Counter()
with tf.Session() as sess:
    for i in range(1000):
        body, length, label, id_ = sess.run(next_element)
        counts[label] += 1
print(counts)

Counter({0: 580, 1: 420})


10007